<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#找有哪些Column要補" data-toc-modified-id="找有哪些Column要補-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>找有哪些Column要補</a></span></li></ul></div>

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import json
import os
import MySQLdb
import getpass
import random
import numpy as np
import re
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd

In [2]:
yr_start = 2023
yr_end = 2023
re_new = True
#----------batter or pitcher-----------
player_types = ['batter', 'pitcher']

In [3]:
#-------------------YWL made password invisible-------------------
import getpass
ID = input('MySQL ID: ')
passwd = getpass.getpass('password: ')
#------------------YWL----------------------------------

MySQL ID: ywliao
password: ········


In [4]:
def dump_players_stats_as_json(yr, player_type, col_list=None):
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"
    headers = {"User-Agent": user_agent}
    if player_type == 'batter':
        url_first_half = f'''https://baseballsavant.mlb.com/leaderboard/custom?year={yr}&\
type={player_type}&filter=&sort=1&sortDir=desc&min=1&selections=pa,'''
        url_second_half = '''&chart=false&x=b_total_pa&y=b_total_pa&r=no&chartType=beeswarm'''
    elif player_type == 'pitcher':
        url_first_half = f'''https://baseballsavant.mlb.com/leaderboard/custom?year={yr}&\
type={player_type}&filter=&sort=1&sortDir=desc&min=1&selections=p_formatted_ip,'''
        url_second_half = '''&chart=false&x=p_formatted_ip&y=p_formatted_ip&r=no&chartType=beeswarm'''
        
    if col_list == None:
        url = url_first_half + url_second_half
    else:
        for col in col_list:
            url_first_half += f'{col},'
        url = url_first_half + url_second_half
#     if player_type == 'batter':
#         url = f'''https://baseballsavant.mlb.com/leaderboard/custom?year={yr}&\
# type={player_type}&filter=&sort=1&sortDir=desc&min=1&selections=pa,&\
# chart=false&x=b_total_pa&y=b_total_pa&r=no&chartType=beeswarm'''
#     elif player_type == 'pitcher':
#         url = f'''https://baseballsavant.mlb.com/leaderboard/custom?year={yr}&\
# type={player_type}&filter=&sort=1&sortDir=desc&min=1&selections=p_formatted_ip,&\
# chart=false&x=p_formatted_ip&y=p_formatted_ip&r=no&chartType=beeswarm'''
    print(url)

    req = requests.get(url=url, headers=headers)
    req.encoding = 'utf-8'
    soup = bs(req.text, 'html.parser')
    table = soup.find('div', {'class': 'article-template'}).script

    table_long_str = '[' + \
        str(table).split('var data = [')[1].split(']')[0] + ']'
    table_json = json.loads(table_long_str)

    return table_json

In [5]:
def dump_mlb_player_stat(yr, player_type):
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"
    headers = {"User-Agent": user_agent}
    if player_type == 'batter':
        url = f'''https://baseballsavant.mlb.com/leaderboard/custom?year={yr}&\
type={player_type}&filter=&sort=1&sortDir=desc&min=1&selections=pa,&\
chart=false&x=b_total_pa&y=b_total_pa&r=no&chartType=beeswarm'''
    elif player_type == 'pitcher':
        url = f'''https://baseballsavant.mlb.com/leaderboard/custom?year={yr}&\
type={player_type}&filter=&sort=1&sortDir=desc&min=1&selections=p_formatted_ip,&\
chart=false&x=p_formatted_ip&y=p_formatted_ip&r=no&chartType=beeswarm'''
    print(url)
    
    req = requests.get(url=url, headers=headers)
    req.encoding = 'utf-8'
    soup = bs(req.text, 'html.parser')
    
    table = soup.find('div',{'class':'article-template'}).script
    table_element = str(table).split(',')
    print(len(table_element))
    
    players = []
    player_name = False
    link = False
    name_display_last_first = False
    #first_player = True
    this_dict = {}
    for this_TE in table_element:
        this_TE = this_TE.replace('}','')
        if this_TE.find('{') >= 0:
            this_TE = this_TE.split('{')[1]
        this_TE2 = this_TE.split(':')
        #if this_TE.find('player_type') >= 0:
        #    if (not first_player):
        #        players.append(this_dict)
        #    #print(this_TE)
        #    this_dict = {}
        #    first_player = False
        if len(this_TE2) == 1 and (not ((player_name or link) or name_display_last_first)):
            break
        if this_TE[1:5] == 'href':
            link = True
            link_line = 0
        if (not (player_name or link)) and (len(this_TE2) == 2):
            this_key = this_TE2[0].replace('"','')
            this_value = this_TE2[1].replace('"','')
            this_dict[this_key] = this_value
            if this_key == 'player_name':
                player_name = True
            if this_key == 'name_display_last_first':
                name_display_last_first = True
        elif player_name:
            this_dict['player_name'] = this_dict['player_name'] + ', ' + this_TE[0:-1]
            player_name = False
        elif name_display_last_first:
            this_dict['name_display_last_first'] = this_dict['name_display_last_first'] + ', ' + this_TE[0:-1]
            name_display_last_first = False
        elif link:
            if link_line == 0:
                link_line = 1
            else:
                players.append(this_dict)
                this_dict = {}
                link = False
        
        
    return players, table_element
    #return players

In [6]:
def mysql_get_one_column(table_name, column_name, data_base = "baseball_stats", constraint_col = "pa", lower_limit = 0, upper_limit = 1e8):
    db = MySQLdb.connect("localhost", ID, passwd, data_base)
    cursor = db.cursor()
    if len(constraint_col) > 0:
        cursor.execute(f'SELECT player_id, {column_name} FROM {table_name} WHERE {constraint_col} BETWEEN {lower_limit} and {upper_limit}')
    else:
        cursor.execute(f'SELECT player_id, {column_name} FROM {table_name}')
    
    id_column = {}
    for player in cursor:
        id_column[player[0]] = player[1]
    return id_column

In [7]:
def show_column_name(table_name, data_base = "baseball_stats", is_num = False):
    db = MySQLdb.connect("localhost", ID, passwd, data_base)
    cursor = db.cursor()
    cursor.execute(f'SHOW FIELDs FROM {table_name}')
    col_list = []
    for row in cursor:
        if is_num:
            if not row[1].find('char') >= 0:
                col_list.append(row[0])
            else:
                print(row)
        else:
            col_list.append(row[0])
    db.close()
    return col_list

In [8]:
def data_type(str):
    try:
        if float(str) % 1 == 0:
            this_type = 'MEDIUMINT'
        else:
            this_type = 'FLOAT'
    except:
        this_type = 'CHAR(30)'
    return this_type

In [9]:
def store_players_into_MySQL_db(players, table_name):
    player_num = len(players)
    
    db = MySQLdb.connect("localhost", ID, passwd, "baseball_stats")
    cursor = db.cursor()
    
    key_list = list(players[0].keys())
    valid_count = np.zeros((len(key_list)))
    key_type = {}
    for player in players:
        for i, key in enumerate(key_list):
            try:
                if not player[key] == 'null':
                    valid_count[i] += 1
                    if key_type.__contains__(key):
                        if not key_type[key] == 'FLOAT':
                            key_type[key] = data_type(player[key])
                    else:
                        key_type[key] = data_type(player[key])
            except:
                pass
    
    valid_key = []
    for i, key in enumerate(key_list):
        if valid_count[i] > 0:
            valid_key.append(key)
    
    for remove_key in ['player_type', 'year', 'yearhidden', 'pid']:
        if remove_key in valid_key:
            valid_key.remove(remove_key)

    sql = f"drop table if exists {table_name}"
    cursor.execute(sql)
    sql = f'create table {table_name} (player_id int(7) not null primary key) default charset = utf8'
    cursor.execute(sql)
    sql = f'ALTER TABLE {table_name} ADD COLUMN player_name char(30)'
    cursor.execute(sql)
    
    for i, key in enumerate(valid_key):
        if not (key == 'player_id' or key == 'player_name'):
            if key == 'double':
                sql_tail = 'double_hit mediumint'
            else:
                sql_tail = key + f' {key_type[key]}'

            sql = f'ALTER TABLE {table_name} ADD COLUMN ' + sql_tail
            cursor.execute(sql)
    
    for i, player in enumerate(players):
        good_data = True
        sql = f'insert into {table_name}(player_id) values({player["player_id"]})'
        try:
            cursor.execute(sql)
            db.commit()
        except:
            print(sql)
            good_data = False
        if good_data:
            for key in valid_key:
                if not key == 'player_id':
                    if not player[key] == None:
                        if key == 'double':
                            sql = f'update {table_name} set double_hit={player["double"]} \
                            where player_id={player["player_id"]}'
                            cursor.execute(sql)
                        else:
                            if not player[key] == 'null':
                                value_str = player[key]
                                if key_type[key][0:4] == 'CHAR':
                                    value_str = '\"' + value_str + '\"'
                                elif value_str == '':
                                    value_str = '0'


                            try:
                                sql = f'update {table_name} set {key}={value_str} \
                                where player_id={player["player_id"]}'
                                #print(sql)
                                cursor.execute(sql)
                            except:
                                print(sql)
            db.commit()
            print(f'Year = {yr}, {i+1}/{player_num}, {player["player_name"]}')
    db.close()

In [10]:
save_path = 'MLB_Player_Stats/'
if not os.path.exists(save_path):
    os.mkdir(save_path)

In [11]:
def find_indices(list_to_check, item_to_find):
    indices = []
    for idx, value in enumerate(list_to_check):
        if value == item_to_find:
            indices.append(idx)
    return indices

def get_column_list(player_type):
    column_name_filename = '欄位分析.xlsx'
    df0 = pd.read_excel(column_name_filename, sheet_name = player_type)
    #print(df0.shape)
    Dict_mark = {df0['欄位名'][i]:df0['Mark'][i] for i in range(df0.shape[0])}
    VA = {df0['欄位名'][i]:df0['變數一'][i] for i in range(df0.shape[0])}
    VB = {df0['欄位名'][i]:df0['變數二'][i] for i in range(df0.shape[0])}
    Operator = {df0['欄位名'][i]:df0['運算子'][i] for i in range(df0.shape[0])}
    col_list_a = [list(Dict_mark.keys())[i] for i in find_indices(list(Dict_mark.values()), 'A')]
    col_list_a2 = [list(Dict_mark.keys())[i] for i in find_indices(list(Dict_mark.values()), 'A2')]
    col_list_f = [list(Dict_mark.keys())[i] for i in find_indices(list(Dict_mark.values()), 'F')]
    col_list_e = [list(Dict_mark.keys())[i] for i in find_indices(list(Dict_mark.values()), 'E')]
    col_list_g = [list(Dict_mark.keys())[i] for i in find_indices(list(Dict_mark.values()), 'G')]
    col_list_y = [list(Dict_mark.keys())[i] for i in find_indices(list(Dict_mark.values()), 'y')]
    col_dict_derived = {col:{'VA':VA[col], 'VB':VB[col], 'Operator':Operator[col]} 
                        for col in col_list_g + col_list_e}
    col_list = {'col_list_a':col_list_a, 'col_list_a2':col_list_a2, 'col_list_f':col_list_f,
               'col_dict_derived':col_dict_derived, 'col_list_y':col_list_y}
    return col_list

In [12]:
def add_columns_for_data_after_2023(player_json, player_type):
    players_df = pd.json_normalize(player_json)
    if player_type == 'batter':
        column_mapping = {'year': 'season_id',
                          'batting_avg': 'ba',
                          'slg_percent': 'slg',
                          'b_hit_by_pitch': 'hbp',
                          'isolated_power': 'iso'}
        players_df['b_inside_park_hr'] = '0'
        hr_flyballs_percent = list(np.array([int(s) for s in players_df['home_run']])\
            / np.array([float(s) + 1e-7 for s in players_df['flyballs']]) * 100)
        hr_flyballs_percent = [f'{i:.1f}' for i in hr_flyballs_percent]
        players_df['hr_flyballs_percent'] = hr_flyballs_percent
    elif player_type == 'pitcher':
        range_breaks = ['sl_range_break_x',
                         'cu_range_break_x',
                         'si_range_break',
                         'ff_range_break',
                         'si_range_break_x',
                         'fc_range_break_z',
                         'kn_range_break',
                         'fs_range_break',
                         'sl_range_break_z',
                         'fs_range_break_z',
                         'ff_range_break_x',
                         'sl_range_break',
                         'fc_range_break',
                         'si_range_break_z',
                         'ch_range_break',
                         'kn_range_break_x',
                         'ch_range_break_z',
                         'fs_range_break_x',
                         'fc_range_break_x',
                         'cu_range_break',
                         'cu_range_break_z',
                         'kn_range_break_z',
                         'ff_range_break_z',
                         'ch_range_break_x']
        column_mapping = {'year': 'season_id',
                          'batting_avg': 'ba',
                          'slg_percent': 'slg',
                          'on_base_percent': 'obp',
                          'isolated_power': 'iso'}
        hr_flyballs_percent = list(np.array([int(s) for s in players_df['home_run']])\
            / np.array([float(s) + 1e-7 for s in players_df['flyballs']]) * 100)
        hr_flyballs_percent = [f'{i:.1f}' for i in hr_flyballs_percent]
        players_df['hr_flyballs_percent'] = hr_flyballs_percent
        
        for col in range_breaks:
            players_df[col] = 'null'
            
        players_df['name_abbrev'] = 'name'
        players_df['hbp'] = players_df['p_hit_by_pitch']
    players_df = players_df.rename(column_mapping, axis=1)
    
    players_json = json.loads(players_df.to_json(orient='records'))
    return players_json

## 找有哪些Column要補

In [13]:
for player_type in player_types:
    for yr in range(yr_start, yr_end+1):
        this_fn = save_path + player_type + '_' + str(yr) + '.json'
        if (not os.path.exists(this_fn)) or re_new:
            players_json = dump_players_stats_as_json(yr, player_type)
            players = add_columns_for_data_after_2023(players_json, player_type)
            with open(this_fn, 'w') as f:
                json.dump(players, f)
            store_players_into_MySQL_db(players, str(yr)+player_type)
        else:
            with open(this_fn, 'r') as f:
                players = json.load(f)

https://baseballsavant.mlb.com/leaderboard/custom?year=2023&type=batter&filter=&sort=1&sortDir=desc&min=1&selections=pa,&chart=false&x=b_total_pa&y=b_total_pa&r=no&chartType=beeswarm
Year = 2023, 1/656, Hernandez, Teoscar
Year = 2023, 2/656, Nootbaar, Lars
Year = 2023, 3/656, Odor, Rougned
Year = 2023, 4/656, Harper, Bryce
Year = 2023, 5/656, Raley, Luke
Year = 2023, 6/656, Hernandez, Enrique
Year = 2023, 7/656, Baty, Brett
Year = 2023, 8/656, Bregman, Alex
Year = 2023, 9/656, Escobar, Eduardo
Year = 2023, 10/656, Bradley Jr., Jackie
Year = 2023, 11/656, Cooper, Garrett
Year = 2023, 12/656, O'Neill, Tyler
Year = 2023, 13/656, Joe, Connor
Year = 2023, 14/656, Monasterio, Andruw
Year = 2023, 15/656, Chapman, Matt
Year = 2023, 16/656, Hampson, Garrett
Year = 2023, 17/656, Ortega, Rafael
Year = 2023, 18/656, Thaiss, Matt
Year = 2023, 19/656, Davis, Jonathan
Year = 2023, 20/656, Davis, J.D.
Year = 2023, 21/656, Margot, Manuel
Year = 2023, 22/656, Haggerty, Sam
Year = 2023, 23/656, Friedl Jr

Year = 2023, 225/656, Alonso, Pete
Year = 2023, 226/656, Gordon, Nick
Year = 2023, 227/656, Toglia, Michael
Year = 2023, 228/656, Julks, Corey
Year = 2023, 229/656, Jeffers, Ryan
Year = 2023, 230/656, Correa, Carlos
Year = 2023, 231/656, Castro, Harold
Year = 2023, 232/656, Fortes, Nick
Year = 2023, 233/656, Cave, Jake
Year = 2023, 234/656, Melendez Jr., MJ
Year = 2023, 235/656, Vargas, Ildemaro
Year = 2023, 236/656, Buxton, Byron
Year = 2023, 237/656, Grissom, Vaughn
Year = 2023, 238/656, Kirilloff, Alex
Year = 2023, 239/656, Castro, Willi
Year = 2023, 240/656, Steer, Spencer
Year = 2023, 241/656, Haniger, Mitch
Year = 2023, 242/656, Zunino, Mike
Year = 2023, 243/656, Bethancourt, Christian
Year = 2023, 244/656, Nola, Austin
Year = 2023, 245/656, Naylor, Josh
Year = 2023, 246/656, Acuna Jr., Ronald
Year = 2023, 247/656, Varsho, Daulton
Year = 2023, 248/656, Vargas, Miguel
Year = 2023, 249/656, Burger, Jake
Year = 2023, 250/656, Volpe, Anthony
Year = 2023, 251/656, Stewart, DJ
Year = 2

Year = 2023, 451/656, Witt Jr., Bobby
Year = 2023, 452/656, Thomas, Lane
Year = 2023, 453/656, Fairchild, Stuart
Year = 2023, 454/656, Benson, Will
Year = 2023, 455/656, Benintendi, Andrew
Year = 2023, 456/656, Goldschmidt, Paul
Year = 2023, 457/656, Caballero, Jose
Year = 2023, 458/656, Wong, Connor
Year = 2023, 459/656, Kiner-Falefa, Isiah
Year = 2023, 460/656, Votto, Joey
Year = 2023, 461/656, Pham, Tommy
Year = 2023, 462/656, Verdugo, Alex
Year = 2023, 463/656, Young, Jacob
Year = 2023, 464/656, Drury, Brandon
Year = 2023, 465/656, Freeman, Freddie
Year = 2023, 466/656, Arias, Gabriel
Year = 2023, 467/656, Tovar, Ezequiel
Year = 2023, 468/656, Perdomo, Geraldo
Year = 2023, 469/656, Semien, Marcus
Year = 2023, 470/656, Bride, Jonah
Year = 2023, 471/656, Larnach, Trevor
Year = 2023, 472/656, Miller, Owen
Year = 2023, 473/656, Palacios, Richie
Year = 2023, 474/656, Carpenter, Kerry
Year = 2023, 475/656, Allen, Nick
Year = 2023, 476/656, Marte, Ketel
Year = 2023, 477/656, Muncy, Max
Ye

Year = 2023, 17/863, Lynch IV, Daniel
Year = 2023, 18/863, Llovera, Mauricio
Year = 2023, 19/863, Pressly, Ryan
Year = 2023, 20/863, Jackson, Jay
Year = 2023, 21/863, Quantrill, Cal
Year = 2023, 22/863, Contreras, Roansy
Year = 2023, 23/863, Littell, Zack
Year = 2023, 24/863, Bielak, Brandon
Year = 2023, 25/863, Hunter, Tommy
Year = 2023, 26/863, Wells, Tyler
Year = 2023, 27/863, Neal, Zach
Year = 2023, 28/863, Skubal, Tarik
Year = 2023, 29/863, Baker, Bryan
Year = 2023, 30/863, Bruihl, Justin
Year = 2023, 31/863, Gray, Jon
Year = 2023, 32/863, Iglesias, Raisel
Year = 2023, 33/863, Faucher, Calvin
Year = 2023, 34/863, Rodriguez, Grayson
Year = 2023, 35/863, Kennedy, Brett
Year = 2023, 36/863, Wilson, Steven
Year = 2023, 37/863, Seabold, Connor
Year = 2023, 38/863, McGough, Scott
Year = 2023, 39/863, Mikolas, Miles
Year = 2023, 40/863, Harvey, Hunter
Year = 2023, 41/863, Okert, Steven
Year = 2023, 42/863, Alexander, Scott
Year = 2023, 43/863, Shreve, Chasen
Year = 2023, 44/863, Sims, Lu

Year = 2023, 247/863, Bard, Daniel
Year = 2023, 248/863, Voth, Austin
Year = 2023, 249/863, Strzelecki, Peter
Year = 2023, 250/863, Lovelady, Richard
Year = 2023, 251/863, Gore, MacKenzie
Year = 2023, 252/863, Brigham, Jeff
Year = 2023, 253/863, Willingham, Amos
Year = 2023, 254/863, Adon, Joan
Year = 2023, 255/863, Balazovic, Jordan
Year = 2023, 256/863, Doyle, Tommy
Year = 2023, 257/863, Diekman, Jake
Year = 2023, 258/863, Darvish, Yu
Year = 2023, 259/863, Ferguson, Caleb
Year = 2023, 260/863, VerHagen, Drew
Year = 2023, 261/863, Smyly, Drew
Year = 2023, 262/863, Bradley, Taj
Year = 2023, 263/863, Henry, Tommy
Year = 2023, 264/863, Castillo, Max
Year = 2023, 265/863, Jansen, Kenley
Year = 2023, 266/863, Houser, Adrian
Year = 2023, 267/863, Kauffmann, Karl
Year = 2023, 268/863, Pagan, Emilio
Year = 2023, 269/863, Junis, Jakob
Year = 2023, 270/863, Kowar, Jackson
Year = 2023, 271/863, Thompson, Ryan
Year = 2023, 272/863, Giolito, Lucas
Year = 2023, 273/863, Gausman, Kevin
Year = 2023, 

Year = 2023, 475/863, Poche, Colin
Year = 2023, 476/863, Gaddis, Hunter
Year = 2023, 477/863, Harris, Hobie
Year = 2023, 478/863, Sandoval, Patrick
Year = 2023, 479/863, Neris, Hector
Year = 2023, 480/863, Edwards Jr., Carl
Year = 2023, 481/863, Bieber, Shane
Year = 2023, 482/863, Soto, Gregory
Year = 2023, 483/863, Houck, Tanner
Year = 2023, 484/863, Cueto, Johnny
Year = 2023, 485/863, Smith, Will
Year = 2023, 486/863, Kluber, Corey
Year = 2023, 487/863, Palencia, Daniel
Year = 2023, 488/863, Freeland, Kyle
Year = 2023, 489/863, Brown, Hunter
Year = 2023, 490/863, Battenfield, Peyton
Year = 2023, 491/863, Alvarado, Jose
Year = 2023, 492/863, Syndergaard, Noah
Year = 2023, 493/863, Snell, Blake
Year = 2023, 494/863, Moreta, Dauri
Year = 2023, 495/863, Carlton, Drew
Year = 2023, 496/863, Rosenberg, Kenny
Year = 2023, 497/863, Uribe, Abner
Year = 2023, 498/863, Brash, Matt
Year = 2023, 499/863, Stroman, Marcus
Year = 2023, 500/863, Frias, Luis
Year = 2023, 501/863, Honeywell Jr., Brent
Y

Year = 2023, 701/863, Gage, Matt
Year = 2023, 702/863, Andrews, Clayton
Year = 2023, 703/863, Sulser, Cole
Year = 2023, 704/863, Padilla, Nicholas
Year = 2023, 705/863, Plassmeyer, Michael
Year = 2023, 706/863, Kelly, Carson
Year = 2023, 707/863, Hendriks, Liam
Year = 2023, 708/863, Woods Richardson, Simeon
Year = 2023, 709/863, Coonrod, Sam
Year = 2023, 710/863, Raley, Luke
Year = 2023, 711/863, Festa, Matt
Year = 2023, 712/863, Ingram, Kolton
Year = 2023, 713/863, Krook, Matt
Year = 2023, 714/863, Young, Danny
Year = 2023, 715/863, Bracho, Silvino
Year = 2023, 716/863, Parsons, Wes
Year = 2023, 717/863, McGee, Easton
Year = 2023, 718/863, Bukauskas, J.B.
Year = 2023, 719/863, Barnhart, Tucker
Year = 2023, 720/863, Wolf, Jackson
Year = 2023, 721/863, Hanifee, Brenan
Year = 2023, 722/863, McAllister, Zach
Year = 2023, 723/863, Sherriff, Ryan
Year = 2023, 724/863, Acton, Garrett
Year = 2023, 725/863, Bristo, Braden
Year = 2023, 726/863, Suero, Wander
Year = 2023, 727/863, Espino, Paolo
